In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png

sys.path.append("../script/")
from MergeBrain import WholeBrainCells
import matplotlib.pyplot as plt
import pandas as pd

import sys, os, os.path, re, csv, math
import linecache
import numpy as  np
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean,stdev
import seaborn as sns

import csv
import pprint
import ants
import tifffile
from multiprocessing import Pool
import datetime
from itertools import product

import numpy as np
import tifffile
import os
import pickle
import joblib
import pandas as pd
from matplotlib import pyplot as plt
import concurrent.futures
import gc
import json

In [2]:
data_GO = pd.read_csv(filepath_or_buffer="/export3/Imaging/atlas_for_fig/GOvsID_6region_removed.csv", encoding="ms932", sep=",")
# data_GO["id"]のユニークな値を取得
ids = data_GO["id"].astype(int)
ids=np.array(ids)
data_GO

,id,graph_order,R,G,B
0,997,0,255,255,255
1,68,7,38,143,69
2,667,8,38,143,69
3,52192,9,38,143,69
4,52196,10,38,143,69
...,...,...,...,...,...
666,129,1299,170,170,170
667,140,1300,170,170,170
668,145,1301,170,170,170
669,153,1302,170,170,170


In [ ]:
conditions = {

    "WT_2m": ["8w_B6JWT_atlas_1_2022_0924_1847","8w_B6JWT_atlas_2_2022_0926_1326", "8w_B6JWT_atlas_3_2022_1005_1110", "8w_B6JWT_atlas_4_2022_1010_2030","8w_B6JWT_atlas_5_2023_0423_1604",],
 
}

In [5]:
dt_voxel = np.dtype([
  ('X(um)', 'f4'), ('Y(um)', 'f4'), ('Z(um)', 'f4'),
  ('NeuN_neighbor_r_100um','f4'), ('atlasID', 'u2')
])

all_paths = [path for paths in conditions.values() for path in paths]
#all_paths = ["1w_B6JWT_atlas_2_2022_1219_1829"]
# 全サンプルのデータを保持するリスト
all_data = []

for Sample in all_paths:
    
    # stitching後 & ANTs後のcellome情報を読み出し
    print("\n")
    start_time = datetime.datetime.now()
    start_time2 = datetime.datetime.now()
    print("Start time:", start_time)
    
    param_path = "/home/mitani/CUBIC-informatics/param/Neuronomics/" + Sample + "/param_multichannel-rank.json"

    print(param_path)



    with open(param_path) as f:
        param = json.load(f)

    fw_dir_nu = param["dst_basedir"]


    ratio_cells = np.fromfile(fw_dir_nu + "/10um_voxel_Neuron_ratio_cellome_2.bin", dtype=dt_voxel)
    
    # 全データリストに追加
    all_data.append(ratio_cells['NeuN_neighbor_r_100um'])
    
# リストをnumpy配列に変換
all_data_array = np.array(all_data)

# 各ボクセルごとの平均を計算
voxel_means = np.mean(all_data_array, axis=0)

# 3D画像の元のサイズ
depth_ori = int(np.round(800 / 1))
height_ori = int(np.round(1320 /1))
width_ori = int(np.round(1140 /1))

# 空の3D配列（画像）を作成
image_3d = np.zeros((depth_ori, height_ori, width_ori), dtype=np.float32)

# nearest_id が 0 より大きい行のみを選択
filtered_df = ratio_cells[voxel_means > 0]

# x, y, z 列を整数のNumPy配列に変換し、インデックス用に使用
z_indices = (filtered_df['Z(um)']* (1/10)).astype(int)
y_indices = (filtered_df['Y(um)']* (1/10)).astype(int)
x_indices = (filtered_df['X(um)']* (1/10)).astype(int)
# IDsをnp.int32型に変換してオーバーフローを防ぐ
ids = voxel_means[voxel_means > 0].astype(np.float32)

# 高度なインデクシングを使用して3D画像にIDを割り当て
image_3d[z_indices, y_indices, x_indices] = ids

# Save the 3D array as a TIFF file
#img_filename_Nuclear_Isocortex_to_ori =   f"/export2/Imaging/ds4_Data4/Cellome/Averaged_Neuron_ratio_10um_within100um_radius.tif"
#tifffile.imsave(img_filename_Nuclear_Isocortex_to_ori, image_3d.astype(np.float32))

#del image_3d, z_indices, y_indices, x_indices, ids
#gc.collect()



Start time: 2024-07-08 18:10:52.844193
/home/mitani/CUBIC-informatics/param/Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847/param_multichannel-rank.json


Start time: 2024-07-08 18:11:02.567893
/home/mitani/CUBIC-informatics/param/Neuronomics/8w_B6JWT_atlas_2_2022_0926_1326/param_multichannel-rank.json


Start time: 2024-07-08 18:11:43.413157
/home/mitani/CUBIC-informatics/param/Neuronomics/8w_B6JWT_atlas_3_2022_1005_1110/param_multichannel-rank.json


Start time: 2024-07-08 18:12:07.273535
/home/mitani/CUBIC-informatics/param/Neuronomics/8w_B6JWT_atlas_4_2022_1010_2030/param_multichannel-rank.json


Start time: 2024-07-08 18:12:33.278749
/home/mitani/CUBIC-informatics/param/Neuronomics/8w_B6JWT_atlas_5_2023_0423_1604/param_multichannel-rank.json


In [22]:
# 3D画像の元のサイズ
depth_ori = int(np.round(800 / 1))
height_ori = int(np.round(1320 /1))
width_ori = int(np.round(1140 /1))

# 空の3D配列（画像）を作成
image_3d_area = np.zeros((depth_ori, height_ori, width_ori), dtype=np.float32)

# nearest_id が 0 より大きい行のみを選択
filtered_df = ratio_cells

# x, y, z 列を整数のNumPy配列に変換し、インデックス用に使用
z_indices = (filtered_df['Z(um)']* (1/10)).astype(int)
y_indices = (filtered_df['Y(um)']* (1/10)).astype(int)
x_indices = (filtered_df['X(um)']* (1/10)).astype(int)
# IDsをnp.int32型に変換してオーバーフローを防ぐ
ids = 1

# 高度なインデクシングを使用して3D画像にIDを割り当て
image_3d_area[z_indices, y_indices, x_indices] = ids

In [24]:
for ii in [np.mean(image_3d[image_3d_area >0]),0.80, 0.60, 0.40, 0.20]:
    # Neuron　ratioの high/lowをわける maskを作製
    thresh = ii #np.mean(image_3d[image_3d_area >0])

    depth_ori = int(np.round(800 / 1))
    height_ori = int(np.round(1320 /1))
    width_ori = int(np.round(1140 /1))

    # 空の3D配列（画像）を作成
    image_3d_mask = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)

    image_3d_mask = image_3d_area.copy()
    image_3d_mask[image_3d > thresh] = image_3d_mask[image_3d > thresh].copy() - image_3d_area[image_3d > thresh]

    # Save the 3D array as a TIFF file
    img_filename_Nuclear_Isocortex_to_ori =   f"/export2/Imaging/ds4_Data4/Cellome/" +str(thresh)+"_Threshed_Mask_WM_ver_Averaged_Neuron_ratio_10um_within100um_radius.tif"
    tifffile.imsave(img_filename_Nuclear_Isocortex_to_ori, image_3d_mask.astype(np.uint16))

In [23]:
thresh = 0.20
image_3d_mask = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)

image_3d_mask = image_3d_area.copy()
print(np.mean(image_3d_mask))
image_3d_mask[image_3d > thresh] = image_3d_mask[image_3d > thresh].copy() - image_3d_area[image_3d > thresh]
print(np.mean(image_3d_mask))

0.4203462
0.083157465


In [11]:
np.min(image_3d)

0.0

In [17]:
thresh = 0.80
image_3d_mask = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)

image_3d_mask = image_3d_area
image_3d_mask[image_3d < thresh] = 0

np.sum(image_3d_mask)/np.sum(image_3d_area)

1.0

In [10]:
print(np.mean(image_3d))

0.18416576


In [5]:
# meanのやつが一番白質っぽいか。
WM_mask = tifffile.imread(f"/export2/Imaging/ds4_Data4/Cellome/0.43812782_Threshed_Mask_WM_ver_Averaged_Neuron_ratio_10um_within100um_radius.tif")

In [6]:
WM_mask.shape

(800, 1320, 1140)

In [7]:
from scipy.ndimage import distance_transform_edt
import datetime

start_time = datetime.datetime.now()
start_time2 = datetime.datetime.now()
print("Start time:", start_time)
        
# 距離変換を実行（1がオブジェクトであるためそのまま使用）
distance_map = distance_transform_edt(WM_mask)

end_time = datetime.datetime.now()
print("Duration:", end_time - start_time)

# 距離マップをtifファイルとして保存
output_path = "/export2/Imaging/ds4_Data4/Cellome/distance_map_using_0.43812782_Threshed_Mask_WM.tif"
tifffile.imwrite(output_path, distance_map.astype(np.float32))

Start time: 2024-07-08 19:59:25.156867


NameError: name 'cou' is not defined

In [8]:
# 距離マップをtifファイルとして保存
output_path = "/export2/Imaging/ds4_Data4/Cellome/distance_map_using_0.43812782_Threshed_Mask_WM.tif"
tifffile.imwrite(output_path, distance_map.astype(np.float32))

In [6]:
# meanのやつが一番白質っぽいか。
# これはGM
GM_mask = tifffile.imread(f"/export2/Imaging/ds4_Data4/Cellome/0.43812782_Threshed_Mask_Averaged_Neuron_ratio_10um_within100um_radius.tif")

In [7]:
from scipy.ndimage import distance_transform_edt
import datetime

start_time = datetime.datetime.now()
start_time2 = datetime.datetime.now()
print("Start time:", start_time)
        
# 距離変換を実行（1がオブジェクトであるためそのまま使用）
distance_map = distance_transform_edt(GM_mask)

end_time = datetime.datetime.now()
print("Duration:", end_time - start_time)

# 距離マップをtifファイルとして保存
output_path = "/export2/Imaging/ds4_Data4/Cellome/distance_map_using_0.43812782_Threshed_Mask_GM.tif"
tifffile.imwrite(output_path, distance_map.astype(np.float32))

Start time: 2024-07-08 20:42:19.760883
Duration: 0:09:49.725601


In [25]:
# 白質の距離を読み出す。
WM_distance = tifffile.imread("/export2/Imaging/ds4_Data4/Cellome/distance_map_using_0.43812782_Threshed_Mask_WM.tif")

In [23]:
depth_ori = int(np.round(800 / 1))
height_ori = int(np.round(1320 /1))
width_ori = int(np.round(1140 /1))

image_3d_mask_ = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)
thresh_step = 20 #10
num = 2 
# 白質の距離を読み出す。
for i in range(num):
    print(i) # maskの外側の層から順に
    min_  = thresh_step * i
    max_ = thresh_step * (i+1)
    output_path = "/export2/Imaging/ds4_Data4/Cellome/"+str(min_)+"_distance_map_using_0.43812782_Threshed_Mask_WM.tif"
    if i < num -1:
        image_3d_mask_[(WM_distance >0) & (WM_distance >= min_) & (WM_distance < max_)] = i
        #tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))
    else:
        image_3d_mask_[(WM_distance >0) & (WM_distance >= min_)] = i

tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))

0
1
2
3


In [22]:
WM_distance.max()

114.61675

In [26]:
# 白質の距離を読み出す。
GM_distance = tifffile.imread("/export2/Imaging/ds4_Data4/Cellome/distance_map_using_0.43812782_Threshed_Mask_GM.tif")

In [24]:
depth_ori = int(np.round(800 / 1))
height_ori = int(np.round(1320 /1))
width_ori = int(np.round(1140 /1))
image_3d_mask_ = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)

thresh_step = 10.0
num = 4 
# 白質の距離を読み出す。
for i in range(num):
    print(i) # maskの外側の層から順に
    #image_3d_mask_ = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)
    min_  = thresh_step * i
    max_ = thresh_step * (i+1)
    output_path = "/export2/Imaging/ds4_Data4/Cellome/"+str(min_)+"_distance_map_using_0.43812782_Threshed_Mask_GM.tif"
    if i < num -1:
        image_3d_mask_[(GM_distance > 0) & (GM_distance >= min_) & (GM_distance < max_)] = i
        #tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))
    else:
        image_3d_mask_[(GM_distance > 0) & (GM_distance >= min_)] = i
        
tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))

0
1
2
3


In [31]:
depth_ori = int(np.round(800 / 1))
height_ori = int(np.round(1320 /1))
width_ori = int(np.round(1140 /1))

image_3d_mask_ = np.zeros((depth_ori, height_ori, width_ori), dtype=np.uint16)
thresh_step = 10 #10
num = 4 
# 白質の距離を読み出す。
for i in range(num):# 灰白質
    print(i) # maskの外側の層から順に
    min_  = thresh_step * i
    max_ = thresh_step * (i+1)
    output_path = "/export2/Imaging/ds4_Data4/Cellome/"+str(min_)+"_distance_map_using_0.43812782_Threshed_Mask_WM_and_GM.tif"
    if i < num -1:
        image_3d_mask_[(GM_distance >0) & (GM_distance >= min_) & (GM_distance < max_)] = 5 - (i+1)
        #tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))
    else:
        image_3d_mask_[(GM_distance >0) & (GM_distance >= min_)] = 5 - (i+1)
        
    print(5 - (i+1))
        
for i in range(num): #白質
    print(i) # maskの外側の層から順に
    min_  = thresh_step * i
    max_ = thresh_step * (i+1)
    output_path = "/export2/Imaging/ds4_Data4/Cellome/"+str(min_)+"_distance_map_using_0.43812782_Threshed_Mask_WM_and_GM.tif"
    if i < num -1:
        image_3d_mask_[(WM_distance >0) & (WM_distance >= min_) & (WM_distance < max_)] = i+1 + 4
        #tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))
    else:
        image_3d_mask_[(WM_distance >0) & (WM_distance >= min_)] = i+1 + 4
        
    print(i+1 + 4)

tifffile.imwrite(output_path.replace("_GM","_GMslice400"), image_3d_mask_[400].astype(np.uint16))
tifffile.imwrite(output_path, image_3d_mask_.astype(np.uint16))

0
4
1
3
2
2
3
1
0
5
1
6
2
7
3
8


In [4]:
# region volume iso 10um を保存する。

# meanのやつが一番白質っぽいか。
annotation_10 = tifffile.imread(f"/export2/Imaging/ds4_Data4/Cellome/30_distance_map_using_0.43812782_Threshed_Mask_WM_and_GM.tif")

non_zero_coords = np.argwhere(annotation_10)
non_zero_values = annotation_10[non_zero_coords[:, 0], non_zero_coords[:, 1], non_zero_coords[:, 2]]

# DataFrameに変換
df_non_zero = pd.DataFrame(non_zero_coords, columns=['z', 'y', 'x'])
df_non_zero['id'] = np.round(non_zero_values)

df_non_zero['z'] = np.round(df_non_zero['z']) #+ 0.5 #* (10/25) # 10のまま.+ 0.5しないと、voxelの中心にないので。
df_non_zero['y'] = np.round(df_non_zero['y']) #+ 0.5 #* (10/25)
df_non_zero['x'] = np.round(df_non_zero['x']) #+ 0.5 #* (10/25)



NameError: name 'mask' is not defined

In [7]:
# マスクを適用した後のid列から、各IDの出現回数を集計
id_counts = df_non_zero['id'].value_counts()

# ユニークなIDリストに基づいて、集計結果をフィルタリング（存在しないIDは0として扱う）
filtered_counts = id_counts.reindex([1,2,3,4,5,6,7,8], fill_value=0)

# DataFrameを作成
df_filtered = filtered_counts.reset_index()
df_filtered.columns = ['atlasID', 'iso_10um_voxel_count']  # 列名を設定

# CSVに保存

csv_file_path =  "/export2/Imaging/ds4_Data4/Cellome/Regional_iso_10um_voxel_count2_WM_and_GM_30_distance_ver.csv"  # 保存するファイルのパス
df_filtered.to_csv(csv_file_path, index=False)